In [ ]:
#PERSONAL MAINTENANCE/PERSONAL CONSUMPTION CALCULATOR
#USE THIS CELL TO CALCULATE THE PERSONAL CONSUMPTION/PERSONAL MAINTENANCE PERCENTAGE. THIS SHOULD BE USED IN WRONGFUL DEATH MATTERS ONLY.
"""
Script: personal_consumption_maintenance_midpoint.py

Purpose:
    Prompt the user for the necessary inputs (income, sex, household size, etc.)
    and compute Personal Consumption (PC) or Personal Maintenance (PM)
    percentages following Eric Christensen’s 2022 methodology.
    Now includes a 'midpoint' option, which is the average of
    the 'low' and 'high' percentage estimates.

References:
    - Christensen, Eric W. (2022).
      “Personal Consumption and Personal Maintenance Estimates Using Empirically Based Expenditure Allocation Rules.”
      Journal of Legal Economics 28(1): 3–46.
"""

# -----------------------
# 1. Dictionary of Regression Coefficients
# -----------------------
# Each entry: (sex, household_size, measure, estimate) -> { 'interview': (aI, bI), 'diary': (aD, bD) }
#    - measure: 'PC' (personal consumption) or 'PM' (personal maintenance)
#    - estimate: 'low' or 'high'
# The "midpoint" will be computed by averaging the final percentages from 'low' and 'high.'
PCPM_COEFFS = {
    # Men, 1-person
    ('male', 1, 'PC', 'high'):   {'interview': (314.942, -0.592683), 'diary': (52.983, -0.528152)},
    ('male', 1, 'PC', 'low'):    {'interview': (235.910, -0.579785), 'diary': (69.088, -0.555610)},
    ('male', 1, 'PM', 'high'):   {'interview': (364.815, -0.610834), 'diary': (38.018, -0.502998)},
    ('male', 1, 'PM', 'low'):    {'interview': (265.456, -0.595023), 'diary': (49.771, -0.530966)},

    # Men, 2-person
    ('male', 2, 'PC', 'high'):   {'interview': (26.768,  -0.515089), 'diary': (452.070, -0.881204)},
    ('male', 2, 'PC', 'low'):    {'interview': (33.911,  -0.554462), 'diary': (452.070, -0.881204)},
    ('male', 2, 'PM', 'high'):   {'interview': (31.796,  -0.533694), 'diary': (761.178, -0.935044)},
    ('male', 2, 'PM', 'low'):    {'interview': (42.678,  -0.579024), 'diary': (761.178, -0.935044)},

    # Men, 3-person
    ('male', 3, 'PC', 'high'):   {'interview': (3.732,   -0.361830), 'diary': (18.579,  -0.610521)},
    ('male', 3, 'PC', 'low'):    {'interview': (5.851,   -0.424854), 'diary': (18.579,  -0.610521)},
    ('male', 3, 'PM', 'high'):   {'interview': (4.399,   -0.379149), 'diary': (19.425,  -0.619607)},
    ('male', 3, 'PM', 'low'):    {'interview': (7.496,   -0.450484), 'diary': (19.425,  -0.619607)},

    # Men, 4-person
    ('male', 4, 'PC', 'high'):   {'interview': (12.086,  -0.476640), 'diary': (7.790,   -0.540042)},
    ('male', 4, 'PC', 'low'):    {'interview': (26.990,  -0.576067), 'diary': (7.790,   -0.540042)},
    ('male', 4, 'PM', 'high'):   {'interview': (14.646,  -0.496820), 'diary': (8.832,   -0.556018)},
    ('male', 4, 'PM', 'low'):    {'interview': (36.770,  -0.607973), 'diary': (8.832,   -0.556018)},

    # Men, 5+ person
    ('male', 5, 'PC', 'high'):   {'interview': (19.658,  -0.540293), 'diary': (11.405,  -0.592661)},
    ('male', 5, 'PC', 'low'):    {'interview': (11.725,  -0.522069), 'diary': (11.405,  -0.592661)},
    ('male', 5, 'PM', 'high'):   {'interview': (24.822,  -0.563766), 'diary': (12.187,  -0.603314)},
    ('male', 5, 'PM', 'low'):    {'interview': (16.141,  -0.554239), 'diary': (12.187,  -0.603314)},

    # Women, 1-person
    ('female', 1, 'PC', 'high'): {'interview': (87.362,  -0.466948), 'diary': (19.026,  -0.431771)},
    ('female', 1, 'PC', 'low'):  {'interview': (80.577,  -0.470534), 'diary': (19.333,  -0.433862)},
    ('female', 1, 'PM', 'high'): {'interview': (97.399,  -0.481773), 'diary': (17.899,  -0.432099)},
    ('female', 1, 'PM', 'low'):  {'interview': (89.788,  -0.485350), 'diary': (18.203,  -0.434306)},

    # Women, 2-person
    ('female', 2, 'PC', 'high'): {'interview': (19.707,  -0.476965), 'diary': (21.774,  -0.581075)},
    ('female', 2, 'PC', 'low'):  {'interview': (21.552,  -0.500580), 'diary': (21.774,  -0.581075)},
    ('female', 2, 'PM', 'high'): {'interview': (22.594,  -0.492073), 'diary': (24.828,  -0.597126)},
    ('female', 2, 'PM', 'low'):  {'interview': (25.624,  -0.519520), 'diary': (24.828,  -0.597126)},

    # Women, 3-person
    ('female', 3, 'PC', 'high'): {'interview': (3.207,   -0.337180), 'diary': (28.552,  -0.627990)},
    ('female', 3, 'PC', 'low'):  {'interview': (5.666,   -0.406188), 'diary': (28.552,  -0.627990)},
    ('female', 3, 'PM', 'high'): {'interview': (3.554,   -0.348881), 'diary': (26.674,  -0.625777)},
    ('female', 3, 'PM', 'low'):  {'interview': (6.605,   -0.423071), 'diary': (26.674,  -0.625777)},

    # Women, 4-person
    ('female', 4, 'PC', 'high'): {'interview': (5.139,   -0.390324), 'diary': (15.001,  -0.575602)},
    ('female', 4, 'PC', 'low'):  {'interview': (7.937,   -0.451890), 'diary': (15.001,  -0.575602)},
    ('female', 4, 'PM', 'high'): {'interview': (6.002,   -0.406818), 'diary': (16.953,  -0.590173)},
    ('female', 4, 'PM', 'low'):  {'interview': (9.933,   -0.475412), 'diary': (16.953,  -0.590173)},

    # Women, 5+ person
    ('female', 5, 'PC', 'high'): {'interview': (12.089,  -0.485617), 'diary': (13.540,  -0.580066)},
    ('female', 5, 'PC', 'low'):  {'interview': (8.653,   -0.479392), 'diary': (13.540,  -0.580066)},
    ('female', 5, 'PM', 'high'): {'interview': (14.756,  -0.505718), 'diary': (14.415,  -0.589079)},
    ('female', 5, 'PM', 'low'):  {'interview': (11.256,  -0.505885), 'diary': (14.415,  -0.589079)},
}


# -----------------------
# 2. Private helper to compute the PC or PM percentage for EXACTLY 'low' or 'high'
# -----------------------
def _compute_single_estimate(income: float, sex: str, household_size: int, measure: str, estimate: str) -> float:
    """
    Internal helper that does a direct dictionary lookup for 'low' or 'high',
    then computes the final consumption/maintenance percentage for that estimate.
    """
    # Ensure household_size is capped at 5 (5 = "5 or more")
    if household_size >= 5:
        household_size = 5

    key = (sex, household_size, measure, estimate)
    if key not in PCPM_COEFFS:
        raise KeyError(f"No regression coefficients available for {key}.")

    coeff_data = PCPM_COEFFS[key]
    aI, bI = coeff_data['interview']
    aD, bD = coeff_data['diary']

    # Use the power-function approach
    return aI * (income ** bI) + aD * (income ** bD)


# -----------------------
# 3. Primary function for user calls
# -----------------------
def get_pcpm_percentage(income: float,
                        sex: str,
                        household_size: int,
                        measure: str = 'PC',
                        estimate: str = 'high'
                       ) -> float:
    """
    Returns the personal consumption (PC%) or personal maintenance (PM%)
    as a percent of total household income, for a given decedent profile.

    :param income: Annual household income in dollars (> 0).
    :param sex: 'male' or 'female' (case-insensitive).
    :param household_size: integer 1..5 (5 means "5 or more").
    :param measure: 'PC' (Personal Consumption) or 'PM' (Personal Maintenance).
    :param estimate: 'low', 'high', or 'midpoint'
                     (the midpoint is the average of the low and high percentages).
    :return: Percentage (0-100) representing the fraction of household income
             that the adult decedent would self-consume (PC or PM).
    """
    # Normalize inputs
    sex = sex.lower().strip()
    measure = measure.upper().strip()
    estimate = estimate.lower().strip()
    if household_size < 1:
        raise ValueError("household_size must be >= 1.")
    if household_size > 5:
        household_size = 5  # treat 5+ as 5
    if income <= 0:
        raise ValueError("Income must be positive.")

    # Validate
    if sex not in ['male','female']:
        raise ValueError("sex must be 'male' or 'female'.")
    if measure not in ['PC','PM']:
        raise ValueError("measure must be 'PC' or 'PM'.")
    if estimate not in ['low','high','midpoint']:
        raise ValueError("estimate must be 'low', 'high', or 'midpoint'.")

    # If the user wants 'midpoint', compute the average of 'low' and 'high'
    if estimate == 'midpoint':
        try:
            val_low = _compute_single_estimate(income, sex, household_size, measure, 'low')
            val_high = _compute_single_estimate(income, sex, household_size, measure, 'high')
            return 0.5 * (val_low + val_high)
        except KeyError:
            raise KeyError(f"Cannot compute 'midpoint' if dictionary entries for 'low' or 'high' are missing.")
    else:
        # 'low' or 'high'
        return _compute_single_estimate(income, sex, household_size, measure, estimate)


# -----------------------
# 4. Main script that prompts the user
# -----------------------
def main():
    print("=== Personal Consumption / Personal Maintenance Calculator (Midpoint Ready) ===\n")

    # 1. Ask user for annual household income
    while True:
        try:
            income_str = input("Enter the ANNUAL household income (e.g., 50000): ")
            income_val = float(income_str)
            if income_val <= 0:
                raise ValueError
            break
        except ValueError:
            print("Invalid input. Please enter a positive number for income.\n")

    # 2. Ask user for the decedent’s sex
    while True:
        sex_str = input("Enter the decedent’s sex (male/female): ").strip().lower()
        if sex_str in ['male','female']:
            break
        else:
            print("Invalid choice. Please type 'male' or 'female'.\n")

    # 3. Ask user for the household size
    while True:
        try:
            hh_str = input("Number of people in household (1..5; use '5' for 5 or more): ")
            hh_val = int(hh_str)
            if hh_val < 1:
                raise ValueError
            if hh_val > 5:
                hh_val = 5  # treat 5+ as 5
            break
        except ValueError:
            print("Invalid input. Enter an integer >= 1.\n")

    # 4. Ask user if they want Personal Consumption or Personal Maintenance
    while True:
        measure_str = input("Do you want Personal Consumption (PC) or Personal Maintenance (PM)? ").strip().upper()
        if measure_str in ['PC','PM']:
            break
        else:
            print("Invalid choice. Please type 'PC' or 'PM'.\n")

    # 5. Ask user for 'low', 'high' or 'midpoint' estimate
    while True:
        estimate_str = input("Choose an estimate: 'low', 'high', or 'midpoint': ").strip().lower()
        if estimate_str in ['low','high','midpoint']:
            break
        else:
            print("Invalid choice. Please type 'low', 'high', or 'midpoint'.\n")

    # ---- Compute the result
    try:
        pct_val = get_pcpm_percentage(
            income=income_val,
            sex=sex_str,
            household_size=hh_val,
            measure=measure_str,
            estimate=estimate_str
        )
    except Exception as e:
        print("\n[ERROR] Could not compute due to:", str(e))
        return

    # 6. Print final
    print("\n-----------------------------------------------------------")
    print(f"Decedent sex: {sex_str}, Household size: {hh_val}")
    print(f"Annual household income = ${income_val:,.2f}")
    print(f"{'Personal Consumption' if measure_str=='PC' else 'Personal Maintenance'} (Measure: {measure_str})")
    print(f"Estimate chosen: {estimate_str}")
    print(f"=> Final Percentage: {pct_val:.2f}% of household income\n")
    print("Multiply this % by the relevant portion of the decedent’s income if you wish to offset earnings.")


# -----------------------
# 5. If run as script, run main()
# -----------------------
if __name__ == "__main__":
    main()


In [ ]:
#Worklife Adjustment Calculator
#Use this cell to calculate the worklife factor, this is the WLE that is divided by the YFS.

# Function to calculate worklife factor as a percentage
def calculate_worklife_factor():
    try:
        # Ask for Worklife Expectancy (WLE) years
        wle_years = float(input("Enter the Worklife Expectancy (WLE) years (can be a decimal): "))

        # Ask for Years to Final Separation (YFS) years
        yfs_years = float(input("Enter the Years to Final Separation (YFS) years (can be a decimal): "))

        # Validate input
        if yfs_years == 0:
            print("YFS years cannot be zero. Please enter a valid number.")
            return

        # Calculate the worklife factor as a percentage
        worklife_factor = (wle_years / yfs_years) * 100

        # Print the result
        print(f"The Worklife Factor is: {worklife_factor:.2f}%")

    except ValueError:
        print("Invalid input. Please enter numeric values for WLE and YFS years.")

# Run the function
if __name__ == "__main__":
    calculate_worklife_factor()


In [ ]:
#AEF CALCULATOR
#Use this cell once all of the factors have been determined, this will provide the total AEF factor needed for the tables

import pandas as pd
import sys
from IPython.display import display

def get_percentage_input(prompt, default=None):
    while True:
        try:
            user_input = input(prompt)
            if user_input.strip() == "" and default is not None:
                return default
            value = float(user_input)
            if 0 <= value <= 100:
                return value
            else:
                print("Please enter a percentage between 0 and 100.")
        except ValueError:
            print("Invalid input. Please enter a numeric value.")

# Initialize variables with default values
gross_earnings_base = 100.0
personal_type = "None"

# Step-by-step user input
gross_earnings_base = get_percentage_input("Enter the Gross Earnings Base (default 100%): ", default=100.0)
worklife_adjustment = get_percentage_input("Enter the Worklife Adjustment (%): ")
unemployment_factor = get_percentage_input("Enter the Unemployment Factor (%): ")
fringe_benefit = get_percentage_input("Enter the Fringe Benefit percentage (%): ")
tax_liability = get_percentage_input("Enter the Tax Liability (%): ")

wrongful_death = input("Is this a wrongful death matter? (yes/no): ").strip().lower()
if wrongful_death == "yes":
    personal_type = input("Is it 'personal maintenance' or 'personal consumption'?: ").strip().lower()
    personal_percentage = get_percentage_input(f"Enter the percentage for {personal_type} (%): ")
else:
    personal_percentage = 0.0

# Perform calculations based on formula
GE = gross_earnings_base
WLE = worklife_adjustment / 100
UF = unemployment_factor / 100
FB = fringe_benefit / 100
TL = tax_liability / 100
PC = personal_percentage / 100

# Applying the formula: AIF = {[((GE x WLE) (1 - UF)) (1 + FB)] - [(GE x WLE) (1 - UF)] (TL)} (1 - PC)
base_adjustment = GE * WLE * (1 - UF)
fringe_adjusted = base_adjustment * (1 + FB)
tax_adjustment = base_adjustment * TL
final_adjusted = (fringe_adjusted - tax_adjustment) * (1 - PC)

total_factor = round(final_adjusted / GE * 100, 2)

# Prepare data for display
data = {
    "Step": [
        "Gross Earnings Base",
        "x WorkLife Adjustment",
        "x (1 - Unemployment Factor)",
        "= Adjusted Base Earnings",
        "x (1 - Tax Liability)",
        "x (1 + Fringe Benefit)",
        "x (1 - Personal Maintenance/Consumption)",
        "= Fringe Benefits/Tax Adjusted Earnings Base",
        "AEF (Adjusted Earnings Factor)",
    ],
    "Percentage": [
        "100.00%",
        f"{worklife_adjustment:.2f}%",
        f"{100 - unemployment_factor:.2f}%",
        f"{base_adjustment:.2f}%",
        f"{100 - tax_liability:.2f}%",
        f"{100 + fringe_benefit:.2f}%",
        f"{100 - personal_percentage:.2f}% ({personal_type.capitalize() if personal_type != 'None' else 'N/A'})",
        f"{fringe_adjusted:.2f}%",
        f"{total_factor:.2f}%",
    ]
}

# Display results in a table format
df = pd.DataFrame(data)
print("\nAdjusted Earnings Factor Calculation:")
print(df.to_string(index=False))

# Save results to a CSV file
csv_save_path = "adjusted_earnings_factor_calculation.csv"
df.to_csv(csv_save_path, index=False)
print(f"\nThe calculation has been saved to '{csv_save_path}'. You can open the file and copy the table as needed.")


In [35]:
#PEDIATRIC WRONGFUL DEATH TABLE GENERATOR

import pandas as pd
import os

# Step 1: Gather user inputs
print("Welcome to the Post-Trial Earnings Calculator!")

# Basic information
first_name = input("Enter the first name: ")
last_name = input("Enter the last name: ")
date_of_birth = input("Enter the date of birth (YYYY-MM-DD): ")
date_of_injury = input("Enter the date of injury (YYYY-MM-DD): ")

# Assumptions
growth_rate = float(input("Enter the future growth rate (as a percentage, e.g., 3.5 for 3.5%): ")) / 100
discount_rate = float(input("Enter the discount rate (as a percentage, e.g., 5 for 5%): ")) / 100
adjustment_factor = float(input("Enter the adjusted earnings factor (e.g., 88.54 for 88.54%): ")) / 100
start_age = int(input("Enter the age of occupational maturity (e.g., 18): "))
wle_years = float(input("Enter the Work Life Expectancy (WLE) in years (e.g., 47.3): "))
starting_wage_base = float(input("Enter the starting wage base (e.g., 46748): "))

# Calculate starting year and portion of the year
start_year = int(date_of_birth.split("-")[0]) + start_age
portion_of_year = round((365 - pd.Timestamp(date_of_injury).dayofyear) / 365, 2)

# Step 2: Initialize the DataFrame
data = {
    "Year": [],
    "Age": [],
    "Portion of Year (%)": [],
    "Wage Base Years": [],
    "Gross Earnings": [],
    "Adjusted Earnings": [],
    "Present Value": []
}

# Step 3: Perform calculations
current_age = start_age
current_year = start_year
wage_base = starting_wage_base
remaining_years = wle_years

while remaining_years > 0:
    # Portion of the year (adjust for decimals in WLE years)
    if remaining_years < 1:
        portion = remaining_years
    else:
        portion = 1.0

    # Gross Earnings
    gross_earnings = wage_base * portion

    # Adjusted Earnings
    adjusted_earnings = gross_earnings * adjustment_factor

    # Present Value
    present_value = adjusted_earnings / ((1 + discount_rate) ** (current_year - start_year + portion))

    # Append data
    data["Year"].append(current_year)
    data["Age"].append(current_age)
    data["Portion of Year (%)"].append(f"{round(portion * 100)}%")
    data["Wage Base Years"].append(f"${wage_base:,.2f}")
    data["Gross Earnings"].append(f"${gross_earnings:,.2f}")
    data["Adjusted Earnings"].append(f"${adjusted_earnings:,.2f}")
    data["Present Value"].append(f"${present_value:,.2f}")

    # Update values
    wage_base *= (1 + growth_rate)
    current_age += 1
    current_year += 1
    remaining_years -= portion

# Step 4: Create the DataFrame
df = pd.DataFrame(data)

# Step 5: Calculate totals
total_future_value = sum(float(value.replace(",", "").replace("$", "")) for value in data["Adjusted Earnings"])
total_present_value = sum(float(value.replace(",", "").replace("$", "")) for value in data["Present Value"])

# Step 6: Export to Excel
file_name = f"{first_name}_{last_name}_Post_Trial_Earnings.xlsx"
file_path = os.path.join(os.getcwd(), file_name)  # Save to the current working directory
with pd.ExcelWriter(file_path) as writer:
    # Write data to the first sheet
    df.to_excel(writer, index=False, sheet_name="Earnings Report")

    # Add totals in a new row below the table
    summary = pd.DataFrame({
        "Year": ["Total"],
        "Age": [""],
        "Portion of Year (%)": [""],
        "Wage Base Years": [""],
        "Gross Earnings": [""],
        "Adjusted Earnings": [f"${total_future_value:,.2f}"],
        "Present Value": [f"${total_present_value:,.2f}"]
    })
    summary.to_excel(writer, index=False, header=False, sheet_name="Earnings Report", startrow=len(df) + 2)

print(f"\nExport successful! The file has been saved as '{file_path}' in your working directory.")


Welcome to the Post-Trial Earnings Calculator!

Export successful! The file has been saved as '/Users/chrisskerritt/EconomicWorkbook/Forte_Dillard Jr._Post_Trial_Earnings.xlsx' in your working directory.
